# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
df.duplicated().sum()

0

Преобразуем тип данных в столбцах "messages" и "calls" к типу int.

In [4]:
df['messages']=df['messages'].astype("int")
df['calls']=df['calls'].astype("int")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


**Вывод** Полученные данные представляют собой таблицу из 3214 строк и 5 столбцов. Явных дубликатов нет. В ходе изучения данных было принято изменить тип двух столбцов для сохраннения памяти.

## Разбейте данные на выборки

Разделим исходные данные на обучающую, валидационную и тестовую выборки. в итоге должны получить соотношение 60/20/20

Разобьем выборки на 60% обучающую и 40% валидационную

In [5]:
features = df.drop('is_ultra', axis=1) # извлекаем признак
target = df['is_ultra'] #цизвлекаем нулевой признак

features_train, features_test, target_train, target_test  = train_test_split(features, 
                                                                             target, 
                                                                             test_size=0.4, 
                                                                             random_state=12345)

Валидационную на тестовую и валидацонную в соотношении 50/50

In [6]:
features_valid, features_test, target_valid, target_test =train_test_split(features_test,
                                                                           target_test,test_size=0.5  , 
                                                                           random_state=12345)

Проверка:

In [7]:
print(f'тренировочная {features_train.shape[0] / len(df):.1%}')
print(f'валидационная {features_valid.shape[0] / len(df):.1%}')
print(f'тестовая {features_test.shape[0] / len(df):.1%}')

тренировочная 60.0%
валидационная 20.0%
тестовая 20.0%


**Вывод:** данные поделены корректно.

## Исследуйте модели

### Логистическая регрессия

In [8]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=10000) 
model.fit(features_train,target_train) # обучите модель на тренировочной выборке
accuracy = model.score(features_valid, target_valid)
print("Качество:", accuracy)

Качество: 0.7107309486780715


### Дерево решений

In [9]:
best_model = None
best_depth = 0
best_accuracy = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345,max_depth=depth)  # инициализируйте модель 
    model.fit(features_train,target_train) # обучите модель на тренировочной выборке
    accuracy = model.score(features_valid, target_valid) 
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        best_depth = depth
print("Глубина дерева:", best_depth,  "Качество:", best_accuracy)        

Глубина дерева: 3 Качество: 0.7853810264385692


### Случайный лес


In [10]:
best_model = None
best_est = 0
best_depth = 0
best_accuracy = 0
mss=2
for est in range(1,15,1):
    for depth in range(1,11):
        for mss in range(2,6):
            model = RandomForestClassifier(random_state = 12345, 
                                           n_estimators = est, 
                                           max_depth = depth,
                                           min_samples_split=mss)
            model.fit(features_train,target_train)
            accuracy = model.score(features_valid, target_valid) 
            if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_model = model
                    best_depth = depth
                    best_est = est
                    best_mss = mss

print("Глубина дерева:", best_depth, "Количество деревьев:",
      best_est, 'Минимальное количество выборок внутреннего узла:', 
      best_mss, "Качество:", best_accuracy) 



Глубина дерева: 7 Количество деревьев: 11 Минимальное количество выборок внутреннего узла: 3 Качество: 0.8055987558320373


*при кол-ве деревьев = 31 качество увеличивалось до 0.809, но было принято решение оставитть качество = 0.806 в пользу меньшего  кол-ва деревьев, чтоб модель не разрасталась*

**Вывод:** В результате исследования модели было выявлено, что самым высоким качеством (аccuracy) обладает модель "RandomForestClassifier" (случайный лес), результат составил 0.806. Наиболее оптимальные гиперпараметры: max_depth=7, min_samples_split=3, n_estimators=11 

Второе место у модели "DecisionTreeClassifier" (дерево решений) показатель аccuracy = 0.785 при max_depth=5

Тройку лидеров закрывает модель "LogisticRegression" (логистическая регрессия) у нее качество 0.711 при solver='lbfgs', max_iter=10000

## Проверьте модель на тестовой выборке

### Логистическая регрессия

In [11]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=10000)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.6842923794712286


### Дерево решений

In [12]:
model = DecisionTreeClassifier(max_depth=4, random_state=1515)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.7744945567651633


### Случайный лес

In [13]:
model = RandomForestClassifier(max_depth=10, n_estimators=10,min_samples_split=4, random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_test, target_test)
print("Качество:", accuracy)

Качество: 0.8087091757387247


**Вывод:** После объективной оценки окончательной модели у нас получились следующие значения метрики Accuracy на тестовой выборке:

Логистическая регрессия -  0.684

Дерево решений - 0.778

Случайный лес - 0.801

## (бонус) Проверьте модели на адекватность

Для проверки моделей используем классификатор DummyClassifier, он сделает прогнозы с использованем  простых правил и не генерирует никаких ценных сведений о данных  

Наиболее эффективно себя показала модель случайного леса - RandomForestClassifier, поэтому для сравнения с результатами DummyClassifier возьмем ее.
         *Аналогично ей можно рассмотреть и DecisionTreeClassifier*


In [14]:
model = DummyClassifier(strategy='prior', random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print('качество с параметром prior:', accuracy)

качество с параметром prior: 0.7060653188180405


In [15]:
model = DummyClassifier(strategy='stratified', random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print('качество с параметром stratified:', accuracy)

качество с параметром stratified: 0.5925349922239502


In [16]:
model = DummyClassifier(strategy='uniform', random_state=12345)
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print('качество с параметром uniform:', accuracy)

качество с параметром uniform: 0.5038880248833593


**Вывод:** После сравнения реального значения качества с фиктивными результатами, можем утверждать, что модели адекватны, так как истинные значения качества больше фиктивных.

## Общий вывод:
**В ходе работы с данными был заменен тип у двух столбцов, больше никаких преобразований не проводилось.**

**Качество (accuracy) - это самый важный критерий для бизнеса: чем выше качество, тем больше прибыли приносит продукт.**

<ul><li>Самое высокое качество у случайного леса, ввиду того, что вместо одного решающего дерева он использует несколько. В нашем случае наилучшее качество составило 0.806 с гипермараметрами: Глубина дерева-7; Количество деревьев-11, минимальное количество выборок -3. На тестовой выборке с глубиной дерева - 10 и кол. деревьев - 10,минимальное количество выборок -4,  аccuracy = 0.801. Так же модель прошла проверку на адекватность.</li>
    
<li>На втором месте — дерево решений. аccuracy = 0.785, глубина дерева = 3. Модель является адекватной и эффективной.</li>
<li>Самое низкое качество предсказания у логистической регрессии. аccuracy = 0.711</li>

*Рекомендации: так как у модели LogisticRegression качество хуже, следует использовать ее в последнюю очередь или не использовать. А предпочтение отдать - RandomForestClassifier*